# Milestone 2; 10,000 battles Project

## Tasks Description

Milestone 2 (20%): the project repo contains a notebook with data collection and descriptive analysis, properly commented, and the notebook ends with a more structured and informed plan for what comes next.

The tasks involving a large amount of data were pre-run and we simply describe their purposes and outputs while showing how to call them in comments.

## Data Collection

In the data collection, one of our main goals is to remain flexible such that we can always go back in our pipeline to fecth more or less data. Thus, our data collection is organised in multiple steps:
1. fetching of all the wikipedia pages of interest
2. fetching of the wanted information on each page
3. fetching of the battle features.

1. fetching of all the wikipedia pages of interest
The data collection is done first on the cluster by using spark jobs in order to read and parse the wikipedia dump.
In this first part, we fetch all the wikipedia pages which have a title corresponding to the following regex: "(((B|b)attle|(S|s)iege) (of|on))".


In [1]:
#TODO run command for pages extraction

#run pages_extraction.py ...

We retrieve 27255 pages with a title satisfying the regex.

2. fetching of the wanted information on each page

We have observed that wikipedia pages about battles often contain an "infobox" summarizing the main information of the battle: title, place, coordinates, dates, casualties, warriers, victory type, etc. An example is observable in https://en.wikipedia.org/wiki/Battle_of_Waterloo.
Thus, our goal is to use these structured data to do our analysis and the next step in our data collection pipeline is the fetch the infoboxes of all the pages we retrieved in the previous step.

In [2]:
#run fields_extraction.py '/datasets/battle-pages-2.json' '/datasets/battle-fields-0.json'

Out of the 27255 pages, 7486 contain an infobox and only 17 contain two infoboxes (these numbers are obtained by using the python file of the next step but are described here for the clarity of the analysis). We consider that we can do a first analysis on the 7486 pages containing an infobox and do not include pages without an infobox or containing multiple infoboxes. These pages are often redirect pages !!!!!!!!!! CHECK OR REMOVE !!!!!!!!!! or pages containing incomplete information.

3. fetching of the battle features.

In this step, we created a python file that allow us to retrieve the features of interest. This allows us to choose which features we want to retrieve and can also help us to divide our future analysis.

In [3]:
#run features_extraction.py '/datasets/battle-fields-0.json' '/datasets/battle-features-0.json' 'title' 'date'

With this last step, we can start our descriptive analysis.

## Descriptive analysis

Our first step is to observe which features are observable for the battles. For example, for how many battles the date or the geographic coordinates are available.

### Available data

In [5]:
import pandas as pd

In [37]:
#run features_extraction.py '../datasets/battle-fields-0.json' 'features_of_interest' 'features_of_interest_count' 'title' 'infobox_0' 'infobox_1' 'date' 'place' coordinates' 'partof' 'casualties1' 'result'

data = pd.DataFrame.from_csv('features_of_interest_count')
data.reset_index(inplace=True)
data.rename(columns={ 'infobox_0' : 'Multiple infoboxes', 'infobox_1' : 'No infobox'}, inplace=True)
data.rename(index={0: 'Counts'}, inplace=True)
data = data.transpose()
data

,Counts
title,27255
Multiple infoboxes,17
No infobox,19752
date,7479
place,7486
coordinates,2124
partof,7279
casualties1,7208
result,7459


As a first analysis, we can see that when an infobox is present, this one is often complete and contains information about most of the battles' features with the exception of coordinates. For coordinates, we will also search the complete wikipedia page or use the place information (name of the city/region) to complete the coordinates when possible.
After this step, we can confirm that we have almost complete information about 7486 battles but we have to see how these information can be used, retrieved and formatted for an explaratory analysis. Thus, we will now observe into details each feature and try to map them to usable data. For missing features, we will also see if we can retrieve these information in the page or using other existing features.

### Features exploration
